# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 9 2022 9:01AM,250373,10,0086152382,ISDIN Corporation,Released
1,Nov 9 2022 9:01AM,250373,10,0086152384,ISDIN Corporation,Released
2,Nov 9 2022 9:01AM,250373,10,0086152415,ISDIN Corporation,Released
3,Nov 9 2022 9:01AM,250373,10,0086152416,ISDIN Corporation,Released
4,Nov 9 2022 9:01AM,250373,10,0086152481,ISDIN Corporation,Released
5,Nov 9 2022 9:01AM,250373,10,0086152482,ISDIN Corporation,Released
6,Nov 9 2022 9:01AM,250373,10,0086152483,ISDIN Corporation,Released
7,Nov 9 2022 9:01AM,250373,10,0086152489,ISDIN Corporation,Released
8,Nov 9 2022 9:01AM,250373,10,0086152493,ISDIN Corporation,Released
9,Nov 9 2022 9:01AM,250373,10,0086152495,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,250367,Released,1
19,250368,Released,18
20,250371,Released,1
21,250372,Executing,1
22,250373,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250367,NaN,NaN,1.0
250368,NaN,NaN,18.0
250371,NaN,NaN,1.0
250372,NaN,1.0,NaN
250373,NaN,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250136,0.0,0.0,24.0
250280,0.0,0.0,44.0
250291,0.0,0.0,1.0
250299,0.0,0.0,1.0
250311,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250136,0,0,24
250280,0,0,44
250291,0,0,1
250299,0,0,1
250311,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250136,0,0,24
1,250280,0,0,44
2,250291,0,0,1
3,250299,0,0,1
4,250311,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250136,,,24
1,250280,,,44
2,250291,,,1
3,250299,,,1
4,250311,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation
23,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc."
24,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc."
25,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation
43,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC"
44,Nov 9 2022 7:24AM,250365,19,"AdvaGen Pharma, Ltd"
53,Nov 8 2022 7:13PM,250364,15,"Virtus Pharmaceuticals, LLC"
67,Nov 8 2022 4:34PM,250362,19,"AdvaGen Pharma, Ltd"
74,Nov 8 2022 4:10PM,250361,16,"Emersa Waterbox, LLC"
75,Nov 8 2022 3:59PM,250359,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation,,,23
1,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc.",,1,
2,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc.",,,1
3,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation,,,18
4,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC",,,1
5,Nov 9 2022 7:24AM,250365,19,"AdvaGen Pharma, Ltd",,,9
6,Nov 8 2022 7:13PM,250364,15,"Virtus Pharmaceuticals, LLC",,,14
7,Nov 8 2022 4:34PM,250362,19,"AdvaGen Pharma, Ltd",,5,2
8,Nov 8 2022 4:10PM,250361,16,"Emersa Waterbox, LLC",,,1
9,Nov 8 2022 3:59PM,250359,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation,23,,
1,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc.",,1,
2,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc.",1,,
3,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation,18,,
4,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC",1,,
5,Nov 9 2022 7:24AM,250365,19,"AdvaGen Pharma, Ltd",9,,
6,Nov 8 2022 7:13PM,250364,15,"Virtus Pharmaceuticals, LLC",14,,
7,Nov 8 2022 4:34PM,250362,19,"AdvaGen Pharma, Ltd",2,5,
8,Nov 8 2022 4:10PM,250361,16,"Emersa Waterbox, LLC",1,,
9,Nov 8 2022 3:59PM,250359,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation,23,,
1,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc.",,1,
2,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc.",1,,
3,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation,18,,
4,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation,23.0,NaN,NaN
1,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc.",NaN,1.0,NaN
2,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation,18.0,NaN,NaN
4,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 9:01AM,250373,10,ISDIN Corporation,23.0,0.0,0.0
1,Nov 9 2022 8:51AM,250372,22,"NBTY Global, Inc.",0.0,1.0,0.0
2,Nov 9 2022 8:51AM,250371,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 9 2022 8:43AM,250368,10,ISDIN Corporation,18.0,0.0,0.0
4,Nov 9 2022 7:44AM,250367,20,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,500741,41.0,0.0,0.0
15,1001098,103.0,0.0,69.0
16,1251723,5.0,0.0,0.0
19,1001317,13.0,5.0,0.0
20,250367,1.0,0.0,0.0
21,751031,3.0,0.0,0.0
22,500743,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,500741,41.0,0.0,0.0
1,15,1001098,103.0,0.0,69.0
2,16,1251723,5.0,0.0,0.0
3,19,1001317,13.0,5.0,0.0
4,20,250367,1.0,0.0,0.0
5,21,751031,3.0,0.0,0.0
6,22,500743,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,0.0,0.0
1,15,103.0,0.0,69.0
2,16,5.0,0.0,0.0
3,19,13.0,5.0,0.0
4,20,1.0,0.0,0.0
5,21,3.0,0.0,0.0
6,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,15,Released,103.0
2,16,Released,5.0
3,19,Released,13.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,69.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,5.0,0.0,0.0,1.0
Released,41.0,103.0,5.0,13.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,69.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,5.0,0.0,0.0,1.0
2,Released,41.0,103.0,5.0,13.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,69.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,5.0,0.0,0.0,1.0
2,Released,41.0,103.0,5.0,13.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()